<!-- SPDX-License-Identifier: CC-BY-4.0 -->
<!-- Copyright Contributors to the ODPi Egeria project 2024. -->

![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/main/assets/img/ODPi_Egeria_Logo_color.png)

### Egeria Workbook

# Loading content packs

## Introduction

A [content pack](https://egeria-project.org/content-packs/) is a collection of curated metadata that can be loaded into your metadata store.  Egeria includes a number of content packs that can be used to explore various features of egeria ot to test your integration code.  This workbook shows how to load a content pack that is stored as a file.  These files have a file extension of `omarchive`.

## Listing the content packs

A list of content packs installed in Egeria's runtime container can be retrieved using the `hey_egeria_cat show list-archives` command or through the `EgeriaTech` python libraries:

----

In [ ]:
import os
view_server = os.environ.get("VIEW_SERVER","qs-view-server")
url = os.environ.get("EGERIA_VIEW_SERVER_URL","https://host.docker.internal:9443")
user_id = os.environ.get("EGERIA_USER", "peterprofile")
user_pwd = os.environ.get("EGERIA_USER_PASSWORD")

from pyegeria import EgeriaTech
import asyncio
import nest_asyncio
nest_asyncio.apply()

from datetime import datetime

egeria_tech = EgeriaTech(view_server, url, user_id, user_pwd)
egeria_tech_token = egeria_tech.create_egeria_bearer_token()

In [ ]:

archiveList = egeria_tech.get_elements_by_property_value("omarchive", ["fileExtension"], "ArchiveFile")
if archiveList:
    print("Known Archive Files:")
    for archiveFile in archiveList:
        if archiveFile:
            properties = archiveFile.get('properties')
            if properties:
               print("* " + properties.get('pathName'))
            

----

A description of these content packs can be found in the [Egeria documentation](https://egeria-project.org/content-packs/).


To load one of these archives, you need to choose a metadata access server where you want the metadata to be stored and then call the load function with the name of the archive.

The code below lists the metadata access servers currently known to Egeria:

----

In [ ]:
serverList = egeria_tech.get_technology_type_elements("Metadata Access Server")

if serverList:
    print("Known Metadata Access Servers:")
    for server in serverList:
        if server:
            guid="???"
            serverName="????"
            description="???????"
            elementHeader = server.get('elementHeader')
            if elementHeader:
                guid = elementHeader.get('guid')
            properties = server.get('referenceableProperties')
            if properties:
                extendedProperties = properties.get('extendedProperties')
                if extendedProperties:
                    serverName=extendedProperties.get('resourceName')
                    description=extendedProperties.get('description')
            print("* " + serverName + " (" + guid + ") - " + description)
            

----

In the code below, the */deployments/content-packs/CocoComboArchive.omarchive* archive is loaded into the *active-metadata-server*.  This provides lots of metadata describing the Coco Pharmceuticals people and operations.  It does not matter if the same archive is loaded multiple times.  This does not result in duplicate versions of the same element.

-----

In [ ]:

egeria_tech.add_archive_file("content-packs/CocoComboArchive.omarchive", None, "qs-metadata-store")

print("Archive loaded!")


----

There is no response, however the archive is loaded if it responds without an error.  If an error comes back indicating that there has been a time out, then the archive will continue to load in the server until it is complete.

The next code loads support for different types of technology.  You can run the cells selectively, or all of them to load all default connectors.

----

In [ ]:

egeria_tech.add_archive_file("content-packs/UnityCatalogContentPack.omarchive", None, "qs-metadata-store")

print("Archive loaded!")


In [ ]:

egeria_tech.add_archive_file("content-packs/PostgresContentPack.omarchive", None, "qs-metadata-store")

print("Archive loaded!")


In [ ]:

egeria_tech.add_archive_file("content-packs/ApacheKafkaContentPack.omarchive", None, "qs-metadata-store")

print("Archive loaded!")


In [ ]:

egeria_tech.add_archive_file("content-packs/ApacheAtlasContentPack.omarchive", None, "qs-metadata-store")

print("Archive loaded!")


In [ ]:

egeria_tech.add_archive_file("content-packs/APIsContentPack.omarchive", None, "qs-metadata-store")

print("Archive loaded!")


In [ ]:

# Postgres is required by the objservability content pack
egeria_tech.add_archive_file("content-packs/PostgresContentPack.omarchive", None, "qs-metadata-store")
egeria_tech.add_archive_file("content-packs/ObservabilityContentPack.omarchive", None, "qs-metadata-store")

print("Archive loaded!")


----

If you want to experiment with a glossary, the Cloud Information Model is a simple commerce glossary.

----

In [ ]:

egeria_tech.add_archive_file("content-packs/CloudInformationModel.omarchive", None, "qs-metadata-store")

print("Archive loaded!")
